In [1]:
#!pip install pymupdf --quiet 

In [2]:
import boto3
import pprint
from botocore.client import Config
import json
import re

import os
import json
import time
import base64
from PIL import Image
import io
from IPython.display import Image
from IPython.display import display
import ipywidgets as widgets
#from skimage import io



pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_runtime = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


In [3]:
from helper.llmcall import * 
model = "anthropic.claude-3-sonnet"


# functionas

In [4]:
# Function to convert image to base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
    return base64_string

# PDF to Image

In [5]:
import fitz  # PyMuPDF, imported as fitz for backward compatibility reasons

file_path = "Image/pge-utility-bill.pdf"
doc = fitz.open(file_path)  # open document
for i, page in enumerate(doc):
    pix = page.get_pixmap()  # render page to an image
    pageindex = f"_{i+1}.png"
    #print(pageindex)
    imagepath = file_path.replace(".pdf",pageindex)
    pix.save(imagepath)


# using GenAI to read image

## sample

In [6]:
# # Example usage
# image_path = 'Image/pge-utility-bill.pdf'  # Specify the path to your image
# base64_string = image_to_base64(image_path)
# Image(filename=image_path) 
# response_text = interactWithLLM("can you describe this image in details?",base64_string,llm_type)
# print('response_text --- \n' + response_text)

## read from folder

In [13]:
directory_path ='Image/electricity'  # Specify the path to your image
total_kwh = 0
for path, folders, files  in os.walk(directory_path):
     for filename in files:
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            base64_string = image_to_base64(image_path)
            Image(filename=image_path) 
            response_text = interactWithLLM(bedrock_runtime,prompt="how many kWh has been used for current period? only mentioned digits. if you do not have it return 0",image=base64_string,llm_type=model)
            print(response_text)   
            print('')

            # Extract all numbers from the output
            numbers = re.findall(r'\b\d{1,3}(?:,\d{3})*\b', response_text)
            
            # Convert the extracted numbers to integers, removing commas
            kwh_values = [int(num.replace(',', '')) for num in numbers]
            
            # Calculate the total kWh
            total_kwh =  total_kwh + sum(kwh_values)

print('\n')
print(f"The total kWh value from all bills is: {total_kwh} kWh")

Image/electricity/e-24sample-bill-for-website.jpg
The number of kWh used for the current period, as shown in the meter reading summary, is 573.

Image/electricity/e-Sample-electric-Bill-2023.jpg
The image shows that the kWh usage for the current billing period is 55 units.

Image/electricity/e-YHU utility bill.png
The total electricity usage in kWh for the current period, as mentioned on the bill, is 1,353.



The total kWh value from all bills is: 1981 kWh


In [8]:
def extract_number(text):
    numbers = re.findall(r'\b\d+(?:,\d+)*(?:\.\d+)?\b', text)
    return float(numbers[0].replace(',', '')) if numbers else 0

def convert_to_gallons(value, unit):
    if 'ccf' in unit.lower() or 'cubic feet' in unit.lower():
        return value * 748  # 1 CCF = 748 gallons
    elif 'gallon' in unit.lower():
        return value
    else:
        return value * 748  # Assume CCF if unit is not specified or is 'unit'

total_gallons = 0

directory_path ='Image/water'  # Specify the path to your image
for path, folders, files  in os.walk(directory_path):
     for filename in files:
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            base64_string = image_to_base64(image_path)
            Image(filename=image_path) 
            response_text = interactWithLLM(bedrock_runtime,prompt="what is the water usage in current period?",image=base64_string,llm_type=model)
            print('response_text --- \n' + response_text)
            # Extract number and unit from response
            value = extract_number(response_text)
            unit = response_text.split()[-1].lower()  # Assume unit is the last word
            
            # Convert to gallons
            gallons = convert_to_gallons(value, unit)
            total_gallons += gallons
            
            print(f"Extracted: {value} {unit}")
            print(f"Converted: {gallons:.2f} gallons\n")

print(f"Total water consumption: {total_gallons:.2f} gallons")

Image/water/w-Water Bill sample image.jpg
response_text --- 
According to the meter reading information on the water bill, the current read is 66 and the previous read is 20. The multiplier is listed as N/A. Therefore, the consumption or water usage for the current billing period is 46 units.
Extracted: 66.0 units.
Converted: 49368.00 gallons

Image/water/w-Water-Bill-06-06-18.jpg
response_text --- 
According to the consumption history table in the image, the water usage for the current billing period (Bill Date 05-17) is 10 units (likely referring to 1,000 gallons or some other unit of water measurement).
Extracted: 5.0 measurement).
Converted: 3740.00 gallons

Image/water/w-waterbill-iowa.png
response_text --- 
According to the utility statement shown in the image, the current water usage reading is 2596 for the service period.
Extracted: 2596.0 period.
Converted: 1941808.00 gallons

Total water consumption: 1994916.00 gallons
